<a href="https://colab.research.google.com/github/pratap977/CS6910/blob/main/Assignment-1/Question%203%2C4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
from keras.datasets import fashion_mnist
import numpy as np
from  matplotlib import pyplot as plt
import time
import math
#import wandb
#wandb.login()





In [ ]:
((xtrain, ytrain), (xtest, ytest)) = fashion_mnist.load_data()

items=["T-shirt/top", "Trouser","Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag","Ankle boot"]
ftrx=np.zeros((60000,784))
ftry=np.zeros((60000,10))
ftex=np.zeros((10000,784))
ftey=np.zeros((10000,10))
for i in range(len(xtrain)):
  ftrx[i]=xtrain[i].flatten();
  ftrx[i]=ftrx[i]/256
  ftry[i][ytrain[i]]=1
for i in range(len(xtest)):
  ftex[i]=xtest[i].flatten();
  ftex[i]=ftex[i]/256
  ftey[i][ytest[i]]=1


     # activation functions and gradients 
def sigmoid(x):
  return np.where(x>=0, 1/(1+np.exp(-x)), np.exp(x)/(1+np.exp(x)))

def grad_sigmoid(z):
  return z*(1-z)

def softmax(z):
  mx=np.max(z)
  z=z-mx
  out=(np.exp(z)+1e-9)/np.sum(np.exp(z)+1e-9)
  return out

def grad_softmax(yh,ty):
  return yh-ty

def Relu(x):
    return np.maximum(0, x)

def grad_Relu(x):
    return [1 if xi>0 else 0 for xi in x]

def Tanh(x):
  a= 2*np.where(x>=0, 1/(1+np.exp(-2*x)), np.exp(2*x)/(1+np.exp(2*x)))
  return a-1

def grad_Tanh(x):
    return 1-Tanh(x)**2


        # parameter initialization
def InitParams(x,init):
    weights=[]
    baises=[]
    if(init=='random'):
      weights.append((np.random.randn(x[0],784)))
      baises.append((np.random.randn(x[0])))
      l=len(x)-1
      for i in range(l):
          weights.append(0.1*(np.random.randn(x[i+1],x[i])))
          baises.append(0.1*(np.random.randn(x[i+1])))
      weights.append(0.1*(np.random.randn(10,x[-1])))
      baises.append(0.1*(np.random.randn(10)))
    else:
      weights.append(np.random.randn(x[0], 784)*np.sqrt(2./(784)))
      baises.append(np.zeros(x[0]))
      l=len(x)-1
      for i in range(l):
          weights.append(np.random.randn(x[i+1], x[i])*np.sqrt(2./(x[i])))
          baises.append(np.zeros(x[i+1]))
      weights.append(np.sqrt(2./(x[-1]))*(np.random.randn(10,x[-1])))
      baises.append((np.zeros(10)))


    return weights,baises


      #  gradient initializations
def initgrads(N,Nunits):
  dh,da,dw,db=[],[],[],[]
  dw.append(np.zeros((Nunits[0],784)))
  db.append(np.zeros(Nunits[0]))
  da.append(np.zeros(Nunits[0]))
  dh.append(np.zeros(Nunits[0]))
  for i in range(1,N):
    dw.append(np.zeros((Nunits[i],Nunits[i-1])))
    db.append(np.zeros(Nunits[i]))
    da.append(np.zeros(Nunits[i]))
    dh.append(np.zeros(Nunits[i]))
  dw.append(np.zeros((10,Nunits[-1])))
  db.append(np.zeros(10))
  da.append(np.zeros(10))
  dh.append(np.zeros(10))
  return dw,db,da,dh



     # Losses and accuracies
def val_acc_loss(W,B,activation,ls_fun):
  cost=0
  count=0
  if (ls_fun=='ce'):
    for i in range(len(ftrx[54000:])):
      ypr,_,_=Feed_Frwd_Nw1(ftrx[i],W,B,activation,ls_fun)
      cost = np.sum(np.multiply(ftry[i],np.log(ypr)))
      if (np.argmax(ypr) ==np.argmax(ftry[i])):
        count+=1
    return -cost, count/6000
    #return np.sum(ay.multiply(np.log(yh)))
  elif (ls_fun=='mse'):
    for i in range(len(ftrx[54000:])):
      ypr,_,_=Feed_Frwd_Nw1(ftrx[i],W,B,activation,ls_fun)
      cost+=np.sum((ftry[i]- ypr)**2)
      if (np.argmax(ypr) ==np.argmax(ftry[i])):
        count+=1
    return cost/6000, count/6000


def test_acc(W,B,activation,ls_fun):
  count=0
  predictions=[]
  matrix=np.zeros((10,10))
  if (ls_fun=='ce'):
    for i in range(len(ftex)):
      ypr,_,_=Feed_Frwd_Nw1(ftex[i],W,B,activation,ls_fun)
      if (np.argmax(ypr) == ytest[i]):
        count+=1
      predictions.append(np.argmax(ypr))
      matrix[np.argmax(ypr)][ytest[i]]+=1
  lables={}
  for i in range(len(items)):
    lables[i]=items[i]
  wandb.log({"confusion matrix": wandb.plot.confusion_matrix(probs=None,
                                                              y_true=ytest,
                                                              preds=predictions,
                                                              class_names=items)})
  wandb.log({"confusion matrix 2": wandb.plots.HeatMap(items, items, matrix, show_text=True) , "Test Accuracy": count/len(ytest)})

 
def Feed_Frwd_Nw1(xTr,Weighs,Bais,activation,loss):
  a_i=[]
  h_i=[]
  a_i.append(np.dot(Weighs[0],xTr)+Bais[0])       
  if (activation=='sig'):
    h_i.append(sigmoid(a_i[0]))
  elif (activation=='tanh'):
    h_i.append(Tanh(a_i[0]))
  elif (activation=='relu'):
    h_i.append(Relu(a_i[0]))
  for i in range(1,len(Weighs)-1):
    a_i.append((np.dot(Weighs[i],h_i[i-1])+Bais[i]))
    if (activation=='sig'):
      h_i.append(sigmoid(a_i[i]))
    elif (activation=='tanh'):
      h_i.append(Tanh(a_i[i]))
    elif (activation=='relu'):
      h_i.append(Relu(a_i[i]))
  a_i.append(np.dot(Weighs[-1],h_i[-1])+Bais[-1])
  h_i.append(softmax(a_i[-1])) 
  yp=h_i[-1]
  return yp,a_i,h_i



def Back_Prop(ip,ypr,ty,a_i,h_i,W,B,N,nrl,activation,Loss):
  k=len(ypr)
  dw,db,da,dh=initgrads(N,nrl)
  if (Loss=="ce"):
    dh[-1]=[-(t/ypr) if t==1 else 0 for t in ty ]
    da[-1]=ypr-ty
  elif (Loss=="mse"):
    dh[-1]=2*(ypr-ty)
    da[-1]=dh[-1]*grad_sigmoid(ypr)

  db[-1]=da[-1] 
  dw[-1]=np.dot((da[-1][:,np.newaxis]),(h_i[-2][:,np.newaxis]).T)
 
  for i in range(N-1,-1,-1):
    dh[i]=np.squeeze(np.dot(W[i+1].T,da[i+1]))

    if (activation=='sig'):
      da[i]=dh[i]*grad_sigmoid(h_i[i])
    elif (activation=='tanh'):
      da[i]=dh[i]*grad_Tanh(a_i[i])
    elif (activation=='relu'):
      da[i]=dh[i]*grad_Relu(a_i[i])
    db[i]=np.copy(da[i])
    if (i==0):
      dw[i]=np.dot(da[i][:,np.newaxis],ip[:,np.newaxis].T)
    else:
      dw[i]=np.dot(da[i][:,np.newaxis],h_i[i-1][:,np.newaxis].T)
  return dw,db


In [ ]:
def gradDecent(ftrx,ftry,init,wd,eta,N,nrl,activation,Ls_fun,epochs,batchsize):
  W,B=InitParams(nrl,init)
  e=0
  while(epochs>0):
    e+=1
    start = time.time()
    epochs-=1
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx[:54000])):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun)
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,W,B,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        for k in range(len(gw)):
          W[k]=np.subtract(W[k],eta*uw[k]/batchsize)
          B[k]=np.subtract(B[k],eta*ub[k]/batchsize)
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k]) 
            
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun)
    #wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  #test_acc(W,B,activation,Ls_fun)  
  return hi,W,B

In [ ]:
nrl=[32] # number of neurons in each hidden layer
N=len(nrl)
a,b,c=gradDecent(ftrx,ftry,'xaviour',0.005,0.15,N,nrl,'tanh','ce',7,1)

epochs : 1 train_loss : 0.1776340251218672   Train_acc: 0.44416666666666665 val_loss: 1.6013826640002866 val_Accuracy: 0.44083333333333335     time: 24
epochs : 2 train_loss : 0.01507765617178558   Train_acc: 0.2639444444444444 val_loss: 1.839265212169281 val_Accuracy: 0.2678333333333333     time: 24
epochs : 3 train_loss : 0.017112116904441848   Train_acc: 0.25551851851851853 val_loss: 5.810422578757502 val_Accuracy: 0.25683333333333336     time: 24
epochs : 4 train_loss : 0.379098367219156   Train_acc: 0.28444444444444444 val_loss: 4.619414106989214 val_Accuracy: 0.2836666666666667     time: 24
epochs : 5 train_loss : 0.02722323879753235   Train_acc: 0.27105555555555555 val_loss: 6.2387669086457995 val_Accuracy: 0.274     time: 24
epochs : 6 train_loss : 0.8691592241190651   Train_acc: 0.32285185185185183 val_loss: 1.4702050170017775 val_Accuracy: 0.3305     time: 24
epochs : 7 train_loss : 0.11481465282265524   Train_acc: 0.2911666666666667 val_loss: 3.7160506475170947 val_Accuracy:

In [ ]:
import copy 
def Momentum(ftrx,ftry,init,batchsize,gamma,wd,eta,N,nrl,activation,Ls_fun,epochs=1):
  W,B=InitParams(nrl,init)
  vw,vb,_,_=initgrads(N,nrl)
  pw,pb,_,_=initgrads(N,nrl)
  e=0
  while(epochs>0):
    e+=1
    start = time.time()
    epochs-=1
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx[:54000])):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun)
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,W,B,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        for k in range(len(gw)):
          # z1,z2=W[k],B[k]
          vw[k]=np.add(gamma*pw[k],eta*uw[k]/batchsize)
          vb[k]=np.add(gamma*pb[k],eta*ub[k]/batchsize)
          W[k]=np.subtract(W[k],vw[k])
          B[k]=np.subtract(B[k],vb[k])
          pw[k]=copy.deepcopy(vw[k])
          pb[k]=copy.deepcopy(vb[k])
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k]) 
            
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun)
    wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    #print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  test_acc(W,B,activation,Ls_fun)  
  return hi,W,B

In [ ]:
import copy 
def Nesterov(ftrx,ftry,init,batchsize,gamma,wd,eta,N,nrl,activation,Ls_fun,epochs=1):
  W,B=InitParams(nrl,init)
  vw,vb,_,_=initgrads(N,nrl)
  pw,pb,_,_=initgrads(N,nrl)
  w,b=copy.deepcopy(W),copy.deepcopy(B)
  e=0
  while(epochs>0):
    start = time.time()
    e+=1
    start = time.time()
    epochs-=1 
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx)):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun) 
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,w,b,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        for k in range(len(gw)):
          # z1,z2=W[k],B[k]
          vw[k]=np.add(gamma*pw[k],eta*uw[k]/batchsize)
          vb[k]=np.add(gamma*pb[k],eta*ub[k]/batchsize)
          W[k]=np.subtract(W[k],vw[k])
          B[k]=np.subtract(B[k],vb[k])
          pw[k]=copy.deepcopy(vw[k])
          pb[k]=copy.deepcopy(vb[k])
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k])
          for k in range(len(dw)):
            w[k]=np.subtract(W[k],gamma*pw[k])
            b[k]=np.subtract(B[k],gamma*pb[k])    
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun)
    wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    #print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  test_acc(W,B,activation,Ls_fun) 
  return hi,W,B

In [ ]:
def rmsprop(ftrx,ftry,init,batchsize,eps,beta,wd,eta,N,nrl,activation,Ls_fun,epochs=1):
  W,B=InitParams(nrl,init)
  vw,vb,_,_=initgrads(N,nrl)
  e=0
  while(epochs>0):
    e+=1
    start = time.time()
    epochs-=1
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx[:54000])):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun)
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,W,B,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        for k in range(len(gw)):
          vw[k]=np.add(beta*vw[k],(1-beta)*uw[k]**2)
          vb[k]=np.add(beta*vb[k],(1-beta)*ub[k]**2)
          W[k]=np.subtract(W[k],((eta/batchsize)/np.sqrt(vw[k]+eps))*uw[k])
          B[k]=np.subtract(B[k],((eta/batchsize)/np.sqrt(vb[k]+eps))*ub[k])
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k]) 
            
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun)
    #wandb.log({"epoch":e,"Train_loss":LOSS/len(ftrx),"Train_acc":acc1/len(ftrx),"val_loss":testloss,"val_Accuracy":testacc})
    print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  #test_acc(W,B,activation,Ls_fun)
  return hi,W,B

In [ ]:
nrl=[128] # number of neurons in each hidden layer
N=len(nrl)
hi,W,B=rmsprop(ftrx,ftry,'xaviour',32,1e-8,0.8,0.00005,0.25,N,nrl,'tanh','ce',epochs=3)

epochs : 1 train_loss : 0.008243925655433198   Train_acc: 0.8264814814814815 val_loss: 0.022521513184138815 val_Accuracy: 0.8301666666666667     time: 73
epochs : 2 train_loss : 0.009505330063839376   Train_acc: 0.844 val_loss: 0.05538573586772259 val_Accuracy: 0.8496666666666667     time: 72
epochs : 3 train_loss : 0.008216197819705874   Train_acc: 0.836 val_loss: 0.041778043652843634 val_Accuracy: 0.8365     time: 73


In [1]:
val=[0.065, 0.06, 0.25, 0.35, 0.00045, 0.00055, 5.5e-06, 0.0001, 1.5e-05, 0.045, 0.05, 0.0035, 6.5e-05, 0.04, 0.0007, 0.00075, 0.035, 0.095, 0.1, 0.0008, 0.0003, 0.025, 4.5e-05, 0.00035, 0.02, 0.085, 0.015, 0.075, 0.08, 0.0009, 9.5e-05, 0.0004, 1e-05, 0.01, 0.009, 0.0095, 0.0085, 0.0075, 0.008, 0.00095, 0.0055, 0.006, 0.0045, 0.005, 0.004, 0.007, 0.003, 0.0025, 0.002, 0.0015, 0.0065]
print(val)

[0.065, 0.06, 0.25, 0.35, 0.00045, 0.00055, 5.5e-06, 0.0001, 1.5e-05, 0.045, 0.05, 0.0035, 6.5e-05, 0.04, 0.0007, 0.00075, 0.035, 0.095, 0.1, 0.0008, 0.0003, 0.025, 4.5e-05, 0.00035, 0.02, 0.085, 0.015, 0.075, 0.08, 0.0009, 9.5e-05, 0.0004, 1e-05, 0.01, 0.009, 0.0095, 0.0085, 0.0075, 0.008, 0.00095, 0.0055, 0.006, 0.0045, 0.005, 0.004, 0.007, 0.003, 0.0025, 0.002, 0.0015, 0.0065]


In [ ]:
s_config = {
                  'method'    : "random",
                  'metric'    : {'name': 'val_Accuracy','goal':'maximize'},
                  'parameters': {
                                  'optimizer'              : {'values': ['rmsprop','adam','nadam']},
                                  'hidden_layer_size'        : {'values':[32, 64,16,20,128,50]},
                                  'num_hidden_layers'                  : {'values':[1,2,3,5,7,8] },
                                  'activation'         : {'values':['sig','relu','tanh']},
                                  'lossfun'            : {'values':['ce']},
                                  'lr'       : {'values':[1e-3,1e-4,1e-2,1e-5,2e-3,2e-4,2e-2,2e-5,3e-3,3e-4,3e-2,3e-5,4e-3,4e-4,4e-2,4e-5,5e-3,5e-4,5e-2,5e-5,6e-3,6e-4,6e-2,6e-5,7e-3,7e-4,7e-2,7e-4,8e-3,8e-4,8e-2,8e-5,9e-3,9e-4,9e-2,9e-5,1e-1,2e-1,1.5e-3,1.5e-4,1.5e-2,1.5e-5,2.5e-3,2.5e-4,2.5e-2,2.5e-5,3.5e-3,3.5e-4,3.5e-2,3.5e-5,4.5e-3,4.5e-4,4.5e-2,4.5e-5,5.5e-3,5.5e-4,5.5e-2,5.5e-5,6.5e-3,6.5e-4,6.5e-2,6.5e-5,7.5e-3,7.5e-4,7.5e-2,7.5e-4,8.5e-3,8.5e-4,8.5e-2,8.5e-5,9.5e-3,9.5e-4,9.5e-2,9.5e-5,1.5e-1,2.5e-1,3.5e-1,4.5e-1,3e-1]},
                                  'weights_initializer': {'values':['random','Xavier']},
                                  'epochs'             :{'values':[5,10,6,8]},
                                  'weightDecay'        :{'values':[0.005,0,0.00005,0.0005]},
                                  'batchsize'         :{'values':[16,32,64,128]},

                                }
                }